In [33]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import re
import os

In [44]:
BASE_DIR = os.path.realpath(os.getcwd()+'../../..')

In [4]:
not_include = ['cover', 'remix', 'live'] # Запрещённые слова для названий песен todo: 'Live'?
max_eps = 1 # Максимальное отклонение от заданной длины дорожки (в секундах)

In [7]:
def secs_to_string(secs):
    m = str(secs//60)
    m = '0'+m if len(m)==1 else m
    s = str(secs%60)
    s = '0'+s if len(s)==1 else s
    return ':'.join([m,s])

In [12]:
def string_to_secs(string):
    m,s = map(int,string.split(':'))
    return m*60+s

In [17]:
def parse_duration(track_html):
    #print 'parsing'
    d = track_html.find('em')
    #print 'asdf'
    while True:
        try:
            d.div.decompose()
        except:
            break
    #print d
    while True:
        try:
            d.a.decompose()
        except:
            break
    #print d
    return d.text.split()[0]

In [32]:
def format_fname(s, space_replacer='_'):
    s = re.sub(r'\(|\)|\'', '', s)
    #for c in ['(',')','\'']:
    #    s = s.replace(c,'')
    s = s.strip(' ')
    s = re.sub(r' ', space_replacer, s)
    return s

In [76]:
def download_file(artist, title, number, href):
    #dname = 'audio/billboard/'+'0'*(4-len(str(number)))+str(number)+'/'
    dname = '/audio/'
    fname = format_fname('{}_{}-{}.mp3'.format(number, artist,title))
    try:
        !mkdir -p {dname}
    except:
        pass
    try:
        !wget -q -nc -t 1 -O {BASE_DIR+dname+fname} '{href}'
    except:
        raise Exception('Found but not downloaded')
        #ds.at[i,'status']='Found but not downloaded'
        #print('Found but not downloaded')
        #continue

In [65]:
list(filter(lambda x: x=='a', 'abba'))

['a', 'a']

In [71]:
def parse_ipleer(artist, title, duration):
    SEARCH_URL = 'https://ipleer.fm/search/q/'
    SONGS_URL = 'https://ipleer.fm'

    url = format_fname('{}{}+{}/'.format(SEARCH_URL, artist, title), space_replacer='+')
    list_page = requests.get(url=url)
    
    if list_page.status_code != 200:
        raise Exception('List_page_not_found')
    
    list_soup = BeautifulSoup(list_page.text, 'html.parser')
    tracks = list_soup.find_all('li',class_='track')
    ntracks = []
    for track_id,track in enumerate(tracks):
        try:
            a = track.find('span',class_='cartist').text
            t = track.find('span',class_='ctitle').text
            d = parse_duration(track)
            h = track.find('a',class_='playlist-down').get('href')
        except:
            a,t,d, h = None, None, None, None
        if a and t and d and h:
            # Если кавер или ремикс (автор и название могут быть перепутаны местами)
            if len(list(filter(lambda x: x in t.lower(), not_include)))>0\
             or len(list(filter(lambda x: x in a.lower(), not_include)))>0:
                continue
            # Если автор не совпадает с заданным (автор и название могут быть перепутаны местами)
            if len(list(filter(lambda x: x in a, artist.split()))) != len(artist.split())\
             and len(list(filter(lambda x: x in t, artist.split()))) != len(artist.split()):
                continue
            # Если название не совпадает с заданным (автор и название могут быть перепутаны местами)
            if len(list(filter(lambda x: x in t, title.split()))) != len(title.split())\
             and len(list(filter(lambda x: x in a, title.split()))) != len(title.split()):
                continue
            try:
                d = string_to_secs(d)-string_to_secs(duration)
            except:
                continue
            if abs(d) > max_eps:
                continue
           
            ntracks.append({'artist':a, 'title':t, 'time_delta':d, 'href':h, 'track':track})
    
    if len(ntracks)==0:
        raise Exception('No_correct_songs_found',url)
    
    ntracks = sorted(ntracks, key= lambda x: x['time_delta'])
    
    dur_eps = 0
    for track in ntracks:
        download_page = requests.get(SONGS_URL+track['href'])
        if download_page.status_code == 200:
            dur_eps = track['time_delta']
            break
    if download_page.status_code != 200:
        raise Exception('Download_page_not_found')
        
    download_soup = BeautifulSoup(download_page.text, 'html.parser')
    download_href = download_soup.find('a',class_='onesongblock-download').get('href')
    
    return download_href, download_page.url, dur_eps

In [72]:
parse_ipleer('Chicago','25 or 6 to 4','4:50')
#print max_eps

('http://data.iplayer.info/file/11040834/ejRaU1hoczJaRGtBT2FjTGpyb0EzaTZhRk1vN3hxQmV2cHJQNDhrck5wM0F4L0hRT21jUjd6QnBkYnBCR3daVEFpUURIZnhRaFR3SlYvT1E5UlR2d3ViN2lMSWV6TzZFWEExWkVGQmJUbGw4azl0aU9yM09uWFczRUJCcThSUzg/Chicago_-_25_or_6_to_4_(iPlayer.fm).mp3',
 'https://ru123.iplayer.info/song/11040834/Chicago_-_25_or_6_to_4/',
 1)

In [79]:
ds = pd.read_csv(BASE_DIR+'/dsets/songs_ds.csv', index_col=0)
ds['href']='nan'
ds['status']='nan'
ds.head()

,title,artist,number,duration,t_eps,href,status
0,Chicago,25 or 6 to 4,176,4:50,NaN,nan,nan
1,Sukiyaki,A Taste Of Honey,1286,3:43,NaN,nan,nan
2,Chiquitita,ABBA,183,5:25,NaN,nan,nan
3,"Knowing Me, Knowing You",ABBA,231,4:04,NaN,nan,nan
4,Honey Honey,ABBA,902,2:54,NaN,nan,nan


In [77]:
ds[ds['status']!='Downloaded'].shape[0]

739

In [52]:
ds[ds['t_eps']<=max_eps].shape[0]

513

In [51]:
ds[ds['status']=='Downloaded'].shape[0]

561

In [53]:
not_load = ds[ds['status']!='Downloaded'].append(ds[ds['t_eps']>max_eps])

In [57]:
not_load[not_load['artist']=='Bananarama'].href

35    https://ipleer.fm/search/q/Bananarama+A+Trick+...
Name: href, dtype: object

In [58]:
not_load[:20]

,title,artist,number,duration,t_eps,href,status
11,Undercover Angel,Alan O'Day,367,3:33,NaN,https://ipleer.fm/search/q/Alan+ODay+Undercove...,No_correct_songs_found
12,Schoool's out,Alice Cooper,195,4:21,NaN,https://ipleer.fm/search/q/Alice+Cooper+Schooo...,No_correct_songs_found
16,Giving You the Best That I Got,Anita Baker,578,4:21,NaN,https://ipleer.fm/search/q/Anita+Baker+Giving+...,No_correct_songs_found
19,I Can't Stand The Rain,Ann Peebles,377,2:34,NaN,NaN,List_page_not_found
20,Could I Have This Dance,Anne Murray,199,5:03,NaN,https://ipleer.fm/search/q/Anne+Murray+Could+I...,No_correct_songs_found
29,Hooked On A Feeling,B.J. Thomas,1048,3:11,NaN,NaN,List_page_not_found
34,"Tarzan Boy (From ""Teenage Mutant Ninja Turtles...",Baltimora,572,3:52,NaN,NaN,List_page_not_found
35,A Trick Of The Night,Bananarama,99,4:09,NaN,https://ipleer.fm/search/q/Bananarama+A+Trick+...,No_correct_songs_found
40,"You're The First, The Last, My Everything",Barry White,796,3:24,NaN,NaN,List_page_not_found
42,Brass Monkey,Beastie Boys,634,2:39,NaN,NaN,List_page_not_found


In [1]:
%%time
#not_load = ds[ds['status']!='Downloaded']

for row in not_load.iterrows():
    i,row = row
    print i,
    if i%5==0:
        ds.to_csv('McGill-Billboard (2)/songs_downloaded.csv')
    
    try:
        download_href, page_url, dur_eps = parse_ipleer(artist=row.artist, title=row.title, duration=row.duration)
        download_file(artist=row.artist, title=row.title, number=row.number, href=download_href)
        
        ds.at[i,'status']='Downloaded'
        ds.at[i,'href']=page_url
        ds.at[i,'t_eps']=dur_eps
        print '+'
    except Exception as e:
        ds.at[i,'status']=e.args[0]
        if len(e.args)==2:
            ds.at[i,'href']=e.args[1]
        print e.args[0]

NameError: name 'not_load' is not defined

In [ ]:
ds.to_csv('McGill-Billboard (2)/songs_downloaded.csv')